In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils,np_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.utils import to_categorical
from keras.datasets import mnist
import pandas as pd


import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [3]:
def KaggleModel(input_shape):
    """
    Implementation of the KaggleModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', padding='same')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    X = Dropout(0.2)(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool0')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv1',padding='same')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X) 
    X = Dropout(0.2)(X)
    
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool1')(X)  
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (7, 7), strides = (1, 1), name = 'conv2', padding = 'same')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)    
    X = Dropout(0.2)(X)    
    
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool2')(X)  

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(10, activation='softmax', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='KaggleModel')
    
    
    ### END CODE HERE ###
    
    return model

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
# Convert one dimentional class arrays to 10-dimentional class matrix
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [6]:
print(y_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [7]:
print('Shape of training data', X_train.shape)
print('Shape of training lables', y_train.shape)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

Shape of training data (60000, 28, 28)
Shape of training lables (60000, 10)


In [8]:
X_train.shape[1:]

(28, 28, 1)

In [9]:
kaggleModel = KaggleModel(X_train.shape[1:])

In [10]:
kaggleModel.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [11]:
kaggleModel.fit(X_train, y_train, batch_size=32, epochs = 10, verbose = 1)

Epoch 1/10
60000/60000 [==============================] - 871s 15ms/step - loss: 0.1258 - acc: 0.9618
Epoch 2/10
60000/60000 [==============================] - 866s 14ms/step - loss: 0.0507 - acc: 0.9840
Epoch 3/10
60000/60000 [==============================] - 873s 15ms/step - loss: 0.0405 - acc: 0.9874
Epoch 4/10
60000/60000 [==============================] - 867s 14ms/step - loss: 0.0321 - acc: 0.9898
Epoch 5/10
60000/60000 [==============================] - 867s 14ms/step - loss: 0.0255 - acc: 0.9923
Epoch 6/10
60000/60000 [==============================] - 869s 14ms/step - loss: 0.0216 - acc: 0.9930
Epoch 7/10
60000/60000 [==============================] - 875s 15ms/step - loss: 0.0187 - acc: 0.9944
Epoch 8/10
60000/60000 [==============================] - 872s 15ms/step - loss: 0.0144 - acc: 0.9954
Epoch 9/10
60000/60000 [==============================] - 869s 14ms/step - loss: 0.0147 - acc: 0.9953
Epoch 10/10
60000/60000 [==============================] - 888s 15ms/step - loss: 

In [12]:
kaggleModel.save('CNN_dropout_kaggle.h5')
print('Model saved to disk successfully.!')

Model saved to disk successfully.!


In [13]:
score = kaggleModel.evaluate(X_test, y_test, verbose=0)
print(score)

[0.024235918441880493, 0.9926]


In [14]:
test = pd.read_csv('test.csv', header = 0) # Loading Kaggle Test set

In [15]:
test_np = test.values

In [16]:
print(test.shape)

(28000, 784)


In [17]:
test_np = test_np.reshape(28000, 28, 28, 1)
prediction = kaggleModel.predict(test_np)
print(prediction.shape)

(28000, 10)


In [18]:
res = {'ImageId' : 'Label'}
for i in range(len(prediction)):
    res[i+1] = np.argmax(prediction[i])

In [19]:
res_np = np.array(list(res.items()))

In [20]:
print(res_np.shape)
print(type(res_np))
print(res_np)

(28001, 2)
<class 'numpy.ndarray'>
[['ImageId' 'Label']
 ['1' '2']
 ['2' '0']
 ...
 ['27998' '3']
 ['27999' '9']
 ['28000' '2']]


In [21]:
df = pd.DataFrame(res_np)
df.to_csv("sub5.csv", index=False, header = False)